In [1]:
import concurrent.futures
from dotenv import load_dotenv
import json
import os
import pandas
import requests
from tqdm.notebook import tqdm as tqdm_notebook

In [ ]:
def chunks(l, n): 
    # looping till length l
    for i in range(0, len(l), n):
        yield l[i:i + n]

In [ ]:
load_dotenv()

BASE_URL = "https://api.github.com/emojis"

GITHUB_API_KEY = os.getenv("GITHUB_API_KEY")
headers = {
    "Authorization": f"Bearer {GITHUB_API_KEY}",
    "Accept": "application/vnd.github+json"
}

In [ ]:
def get_emoji(key : str, url : str):
    response_image = requests.get(url)
    emoji_file = open(f"./images/{key}.png", "wb")
    emoji_file.write(response_image.content)
    emoji_file.close()
    return "OK"

response = requests.get(f"{BASE_URL}", headers=headers)
if response.status_code == 200:
    emojis_dict = response.json()
    keys = list(emojis_dict.keys())
    emojis = []
    for key in keys:
        value = emojis_dict[key]
        emojis.append({
            "name": key,
            "icon": value
        })
    emojis_data_frame = pandas.DataFrame(emojis)
    emojis_data_frame.to_csv("./emojis.csv", index=False, header=True)
    emojis_json = json.dumps(emojis, indent=2)
    emojis_json_file = open("./emojis.json", "w+")
    emojis_json_file.write(emojis_json)
    with concurrent.futures.ThreadPoolExecutor() as executor:
        futures = []
        for key in keys:
            url = emojis_dict[key]
            futures.append(executor.submit(get_emoji, url=url, key=key))
        for future in concurrent.futures.as_completed(futures):
            status = future.result()